<a href="https://colab.research.google.com/github/dldbwls00/NLP-sentiment-analysis-project/blob/main/kobert_sentiment_0109ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kobert finetuning

In [1]:
# package install
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install tokenizers
!pip install torch
!pip install pandas
!pip install transformers==4.30.0
!pip install gluonnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 27.1 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661778 sha256=257a16c980959cdcde8f594bd4c39cba0ed24392a6f3901d93b1822434a5e103
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uni

In [2]:
# kobert install
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-0ia4suaz/kobert-tokenizer_5602c1a010c24b66a427b68ed26dc20b
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-0ia4suaz/kobert-tokenizer_5602c1a010c24b66a427b68ed26dc20b
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=26f85b38f0df18ecc53e35f50a5801da1a169d14fb2be6c183037c93109e6451
  Stored in directory: /tmp/pip-ephem-wheel-cache-jouxlagf/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [4]:
# importing kobert using hugging face
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
# gpu setting
#device = torch.device("cuda:0")
#!nvidia-smi
device = torch.device("cpu")

In [6]:
# tokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [7]:
# connecting google drive - colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# loading dataset
import pandas as pd
conv_training = pd.read_excel('/content/drive/MyDrive/dataset/conv_training.xlsx')
#conv_validation = pd.read_excel('/content/drive/MyDrive/dataset/conv_validation.xlsx')

In [9]:
# dataset check - training
print(len(conv_training))
conv_training.sample(n=5)

51630


,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
37867,37868,청소년,남성,가족관계,해당없음,분노,노여워하는,진짜 가족이라지만 너무 해. 할머니는 매일 내가 여자라는 이유로 공부한다는 나한테 ...,무슨 문제 있으신가요?,남동생이 해야하는 것까지 나한테 지시하잖아!,애타시겠어요. 내 일도 아닌데 신경쓰이시고요.,NaN,NaN
49753,49754,노년,남성,대인관계,만성질환 무,슬픔,우울한,이제 나이가 들어서 회사에 내 편도 없고 퇴사를 하자니 돈도 없어서 정말 눈물이 나.,경제적인 일을 해야 하는데 회사에도 편이 없어서 걱정되시겠어요.,당장 돈을 벌어야 살아갈 수 있는 상황이라 정말 막막해.,이런 상황 속에서 우울함을 덜 느끼는 데 도움이 될 만한 것이 있을까요?,나는 노래를 잘하니 노래로 돈을 벌 수 있는 일을 찾아봐야겠어.,꼭 맞는 일을 찾으셔서 우울함이 사라지면 좋겠어요.
22413,22414,청년,여성,"연애,결혼,출산",해당없음,슬픔,낙담한,남자 친구는 심심할 때나 외로울 때만 나를 찾는 것 같아서 속상해.,남자 친구가 자기 필요할 때만 찾아서 괴로우시군요.,내가 필요할 때는 바쁘다는 핑계만 대는데 정말 화가 나더라고.,기분이 많이 안 좋으시겠어요. 어떻게 하면 지금 상황을 변화시킬 수 있을까요?,남자 친구에게 내 속마음을 진지하게 얘기해봐야겠어.,남편과 이야기해서 좋은 방향으로 이야기가 되었으면 좋겠네요.
13248,13249,청년,여성,대인관계,해당없음,기쁨,안도,대학교수님이 갑자기 나를 부르셔서 긴장했는데 칭찬을 받아서 기뻤어.,칭찬을 받으셔서 기쁘시겠네요.,응. 엄하기로 유명한 교수님인데 칭찬을 해주셨거든.,엄한 교수님에게 좋은 인상을 보이기 위해서는 어떻게 해야 할까요?,지금 내가 하는 거보다 더 열심히 하면 교수님이 나를 더 예쁘게 봐주시지 않을까?,대학교수님께 좋은 인상을 보이시길 바랄게요.
24421,24422,노년,남성,"건강,죽음",만성질환 무,불안,초조한,아내가 곧 임종할 것 같다고 하는데 자식들이 아직 오지 않았어. 마음이 급해.,아직 도착하지 않은 자녀분들 때문에 초조하시군요.,이러다가 자식들이 아내의 임종을 보지 못할까 봐 불안해.,많이 불안하시군요. 어떻게 하면 상황이 해결될까요?,다시 한 번 연락을 해봐야겠어. 빨리 전화를 받아주면 좋겠는데.,아내 분과 이야기를 하셔서 잘 해결하시면 좋겠네요.


In [10]:
# dataset check - validation
#print(len(conv_validation))
#conv_validation.sample(n=5)

In [11]:
# converting label
conv_training.loc[(conv_training['감정_대분류'] == '기쁨'), '감정_대분류'] = 0
conv_training.loc[(conv_training['감정_대분류'] == '당황'), '감정_대분류'] = 1
conv_training.loc[(conv_training['감정_대분류'] == '분노'), '감정_대분류'] = 2
conv_training.loc[(conv_training['감정_대분류'] == '불안'), '감정_대분류'] = 3
conv_training.loc[(conv_training['감정_대분류'] == '상처'), '감정_대분류'] = 4
conv_training.loc[(conv_training['감정_대분류'] == '슬픔'), '감정_대분류'] = 5

In [12]:
# drop NaN + merge
conv_training['사람문장1'] = conv_training['사람문장1'].replace(pd.np.nan, '')
conv_training['사람문장2'] = conv_training['사람문장2'].replace(pd.np.nan, '')
conv_training['사람문장3'] = conv_training['사람문장3'].replace(pd.np.nan, '')
conv_training['sentences'] = conv_training['사람문장1'] + " " + conv_training['사람문장2'] + " " + conv_training['사람문장3']

<ipython-input-12-3ccb88233337>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  conv_training['사람문장1'] = conv_training['사람문장1'].replace(pd.np.nan, '')
<ipython-input-12-3ccb88233337>:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  conv_training['사람문장2'] = conv_training['사람문장2'].replace(pd.np.nan, '')
<ipython-input-12-3ccb88233337>:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  conv_training['사람문장3'] = conv_training['사람문장3'].replace(pd.np.nan, '')


In [13]:
# dataset check - preprocessing conv_training
extracted = pd.DataFrame({
        'emotion': conv_training['감정_대분류'],
        'sentences': conv_training['sentences']
        })

print(len(extracted))
extracted.sample(n=5)

51630


,emotion,sentences
15416,5,고등학교에서 만난 소꿉친구가 날 괴롭히는데 앞장섰어. 너무 속상해. 이대로라면 계속...
46261,5,부모님도 모시면서 아이들을 키우기는 쉽지 않은 것 같아. 아내도 지친 것 같고. 매...
34652,0,작은아들이 이번에 공무원 시험에서 떨어졌어. 하지만 나는 꼭 합격할 거라고 믿어. ...
39314,2,요즘 나 너무 힘들어. 노년기가 되니 남편이 치매가 생겨서 이혼하고 싶어. 늙어서 ...
17064,4,병원에서 입원비를 내지 못하면 더 치료를 해주지 않을 거래. 치료를 받아야 하는데 ...


In [14]:
# export file
#extracted.to_excel('extracted.xlsx')

In [15]:
# extract column -> to array
data_list = []
for q, label in zip(extracted['sentences'], extracted['emotion']) :
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [16]:
# array check
print(len(data_list))
print(data_list[0])
print(data_list[10000])
print(data_list[20000])
print(data_list[30000])
print(data_list[40000])
print(data_list[50000])
print(data_list[-1])

51630
['일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고. ', '2']
['나는 늘 모두에게 사랑받아야 한다고 생각하면서 살아왔어. 오늘 아침에 엄마랑 싸웠는데 엄마가 더 나를 사랑하지 않을까 봐 무서워.  아니 그러지는 않았어. ', '3']
['오늘 갑자기 사장님이 직장을 그만두라고 하셨어. 회사 상황이 좋지 못해서 한 명씩 사직시키고 있다는데 배신당한 기분이야. 부당해고로 신고할까 해.', '4']
['이번에 주식을 샀는데 주가가 너무 떨어져 손해가 이만저만이 아냐. 지금은 없어. 난 왜 이렇게 되는일이 없나 몰라. ', '5']
['다리 절뚝거린다고 내가 이상한 사람도 아닌데 말이야. 친구들도 멀리하니 슬프네! 젊은 시절 일하다가 다리를 다쳤는데 완전히 치료하지 못했어. 친구들도 자식들도 나를 창피해하는 거 같아. 억울하지 내가 피해를 준 것도 없는데 말이야. 주변 사람들 때문에 더 외롭고 힘들어.', '5']
['남편이 몸이 아파서 입원을 했는데 나도 몸이 성치 않은데 병간호를 해야 해서 힘들어. 슬프네. 돈이라도 많으면 간병인을 쓰겠는데 이것도 여의치 않고 이러다 나도 병나겠어. 나라도 몸이 아프지 않게 잘 먹고 몸 관리를 잘해야겠어.', '5']
['남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져. 사람들을 만나는 것이 어려워. 자꾸 사람들을 의심하게만 되고 말이야. 사람들을 볼 때 의심하고 불신하는 마음을 억눌러야겠어. 사람들을 색안경을 끼고 보지 않게.', '4']


In [17]:
# splitting data
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(len(dataset_train))
print(len(dataset_test))

38722
12908


In [18]:
"""class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
"""


class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [19]:
# setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [20]:
# preprocessing
tok = tokenizer.tokenize
#tokenizer = get_tokenizer()
#tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [21]:
# kobert training model
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [22]:
# load model
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [23]:
# prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:
# model training
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-24-4b508a570d4d>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/606 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8008227348327637 train acc 0.15625
epoch 1 batch id 201 loss 1.3312712907791138 train acc 0.30783582089552236
epoch 1 batch id 401 loss 0.9911055564880371 train acc 0.45078709476309226
epoch 1 batch id 601 loss 0.8817806839942932 train acc 0.5113092762063228
epoch 1 train acc 0.5118605610561056


<ipython-input-24-4b508a570d4d>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/202 [00:00<?, ?it/s]

...

epoch 4 test acc 0.6686332695769578


  0%|          | 0/606 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.4777405560016632 train acc 0.828125
epoch 5 batch id 201 loss 0.5489781498908997 train acc 0.7925995024875622
epoch 5 batch id 401 loss 0.5873135328292847 train acc 0.8081748753117207
epoch 5 batch id 601 loss 0.3027040362358093 train acc 0.8130199667221298
epoch 5 train acc 0.8130414603960396


  0%|          | 0/202 [00:00<?, ?it/s]

epoch 5 test acc 0.6701029477947795
